In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install face_recognition
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_vggface
!pip install keras_applications
!pip install keras==2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=37566e128cb6289a8d2f6efe4f18bb2b60672c3fff5e9c0920b4d98d01c25e9c
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-pfovxsxg
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-pfovxsxg
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8294 sha256=837f914d75273a5512c951ead711b4666cae182bdd2e85f3306a2774902bc5b8
  Stored in 

In [3]:
import os
os.chdir('/content/drive/MyDrive/DeepLearning/Research/Face_Recognition')

In [5]:
import tensorflow as tf

print(tf.__version__)  # TensorFlow version


2.16.1


In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/keras-vggface')

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras_vggface.vggface import VGGFace

In [5]:

base_model = VGGFace(include_top=False,model='senet50',input_shape=(224, 224, 3))
base_model.summary()

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# final layer with softmax activation
preds = Dense(128)(x)

104944616/104944616 [==============================] - 0s 0us/step
Model: "vggface_senet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)          

In [ ]:
# # create a new model with the base model's original input and the new model's output
# model = Model(inputs = base_model.input, outputs = preds)
# model.summary()

# # don't train the first 22 layers - 0..22
# for layer in model.layers[:22]:
#     layer.trainable = False

# # train the rest of the layers - 22 onwards
# for layer in model.layers[22:]:
#     layer.trainable = True

In [ ]:
model.compile(optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
# creates a HDF5 file
model.save('senet50_transfer_learning_trained_face_cnn_model.h5')

In [ ]:
# # deletes the existing model
# del model

In [ ]:
from keras.models import load_model
model = load_model('./save_models'
                    +'senet50_transfer_learning_trained_face_cnn_model.h5')

In [ ]:
import cv2
import os
import pickle
import numpy as np
import pickle

from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras_vggface import utils

# dimension of images
image_width = 224
image_height = 224

In [ ]:
# chạy feature extraction trên tập huấn luyện
import tensorflow as tf
import face_recognition
import os

encodeListKnow = []
classnames = []
dem = 0
folder_path = f'/content/drive/MyDrive/Colab Notebooks/keras-vggface/HuanLuyen'

for root, dirs, files in os.walk(folder_path):
  name = root.split("/")[-1]
  for file in files:
      if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg") or True:

          test_image_filename = os.path.join(root, file)
          namecorrect = file.split(" ")[0]

          imagetest = face_recognition.load_image_file(test_image_filename)
          imagetest = cv2.cvtColor(imagetest, cv2.COLOR_BGR2RGB)

          # detect face
          faces = face_recognition.face_locations(imagetest)

          # if not exactly 1 face is detected, skip this photo
          if len(faces) == 0:
              print(f'---We need exactly 1 face; photo skipped---')
              continue
          dem += 1
          print(dem)
          for (y1, x2, y2, x1) in faces:
              # resize the detected face to 224x224
              size = (image_width, image_height)

              # detected face region
              roi = imagetest[y1:y2, x1:x2]

              # resize the detected head to target size
              resized_image = cv2.resize(roi, size)

              # prepare the image for prediction
              x = tf.keras.preprocessing.image.img_to_array(resized_image)
              x = np.expand_dims(x, axis=0)
              x = utils.preprocess_input(x, version=1)

              # making prediction
              predicted_prob = model.predict(x)
              print(predicted_prob)
              # print(predicted_prob[0].argmax())
              encode = predicted_prob[0]
              encodeListKnow.append(encode)
              classnames.append(name)
          print("------------------------------------------------------------------------------\n")

# save encodeListKnow + classnames in file know_face_encode.csv
# Create a DataFrame to store the face encodings
df = pd.DataFrame(encodeListKnow)
dfname = pd.DataFrame(classnames)

# Save the DataFrame to a CSV file
df.to_csv("/content/drive/MyDrive/Colab Notebooks/keras-vggface/SaveDir/know_face_encode.csv", index=False)
dfname.to_csv("/content/drive/MyDrive/Colab Notebooks/keras-vggface/SaveDir/know_face_classnames.csv", index=False)

1
1/1 [==============================] - 12s 12s/step
[[ 3.3374686e+00  5.5347300e+00 -1.9743775e+00 -1.7568209e+00
   4.9583793e-02 -4.0050559e+00 -1.7344552e+00 -2.6400054e+00
  -8.1970751e-01  1.5828013e-01  4.0442686e+00  2.4051416e+00
  -2.0107961e-01  4.8433614e-01  2.3697765e+00  1.0580168e+00
   4.8635702e+00 -1.9477684e+00  1.9108433e+00 -1.3403463e+00
   4.1890202e+00 -5.8355176e-01 -3.8502216e-01  1.9714696e+00
  -7.3333275e-01  2.7675378e+00 -2.1719172e+00  1.5425162e+00
  -1.3074787e+00 -5.7820690e-01 -4.0007038e+00 -3.0386633e-01
   2.1978021e+00 -1.0261824e+00  3.0779523e-01  5.7654362e+00
   1.6598892e-01  4.0228381e+00  1.1287967e+00 -1.5175540e+00
  -7.5135517e-01  3.5513444e+00 -3.4177725e+00 -2.6309988e+00
  -1.0349529e+00 -2.4006414e+00 -3.0246890e-01 -1.5475160e+00
   4.8887360e-01 -1.0405884e+00 -3.0931132e+00  2.5399697e-01
   7.7137589e-01  1.5372965e+00 -2.0910339e+00  1.7623315e+00
   1.9172068e+00 -8.4808445e-01 -1.3468800e+00  2.3180056e+00
  -3.6620674e+00

In [ ]:
import tensorflow as tf
import face_recognition
import os

encodeListUnknow = []
classnamesCorrect = []

folder_path = f'/content/drive/MyDrive/Colab Notebooks/keras-vggface/KiemThuUnknow'
for file in os.listdir(folder_path):
    if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg"):

        test_image_filename = os.path.join(folder_path, file)
        namecorrect = file.split(" ")[0]
        imagetest = face_recognition.load_image_file(test_image_filename)
        imagetest = cv2.cvtColor(imagetest, cv2.COLOR_BGR2RGB)

        # detect face
        faces = face_recognition.face_locations(imagetest)

        # if not exactly 1 face is detected, skip this photo
        if len(faces) == 0:
            print(f'---We need exactly 1 face; photo skipped---')
            continue

        for (y1, x2, y2, x1) in faces:
            # resize the detected face to 224x224
            size = (image_width, image_height)

            # detected face region
            roi = imagetest[y1:y2, x1:x2]

            # resize the detected head to target size
            resized_image = cv2.resize(roi, size)

            # prepare the image for prediction
            x = tf.keras.preprocessing.image.img_to_array(resized_image)
            x = np.expand_dims(x, axis=0)
            x = utils.preprocess_input(x, version=1)

            # making prediction
            predicted_prob = model.predict(x)
            print(predicted_prob)
            # print(predicted_prob[0].argmax())
            encode = predicted_prob[0]
            encodeListUnknow.append(encode)
            classnamesCorrect.append(namecorrect)
        print("------------------------------------------------------------------------------\n")

# save encodeListKnow + classnames in file know_face_encode.csv
# Create a DataFrame to store the face encodings
df = pd.DataFrame(encodeListUnknow)
dfname = pd.DataFrame(classnamesCorrect)

# Save the DataFrame to a CSV file
df.to_csv("/content/drive/MyDrive/Colab Notebooks/keras-vggface/SaveDir/unknow_face_encode.csv", index=False)
dfname.to_csv("/content/drive/MyDrive/Colab Notebooks/keras-vggface/SaveDir/unknow_face_classnames.csv", index=False)

1/1 [==============================] - 0s 26ms/step
[[ 0.9842035   4.8315034  -1.7327471  -3.9380386   0.04767501 -5.696102
  -1.7442676  -2.7232385  -4.784997    1.4918072   5.653989    2.6202776
   0.19967008  2.8471608   1.8720899   3.962819    5.6749964  -3.857277
   3.5025842  -1.661974    4.927126    0.65950596 -0.74777305  2.9277065
  -1.8705983   5.838268   -1.4248717   0.9187844  -1.6934085  -0.280344
  -2.9024177   1.3961303   2.116413   -0.97366166 -0.21213388  5.047965
  -1.1326572   2.438178    2.1310325   0.40990293 -2.456055    3.5925121
  -2.7351227  -4.727922   -3.3298643  -3.9322784  -1.948587   -0.8640326
   0.04908866 -2.2044835  -3.6515121  -1.3191291   2.1960669   1.5958489
  -0.07793057  2.2329116   2.238473   -1.0888779  -1.2757683   3.5070329
  -3.5548015   0.5203843   3.9824646   2.3744607   1.2983046   2.5285776
   5.0254707  -0.26255238  0.20940483  1.7880895   1.7334096   1.9644907
   2.404747   -2.0741675  -0.57447743  2.3451734   0.6658978  -1.5484849
   

In [ ]:
# distance.py
import face_recognition
import numpy as np
from datetime import datetime
import pandas as pd
import os

#path main
pathmain = '/content/drive/MyDrive/Colab Notebooks/keras-vggface/SaveDir'

# read encodeListKnow + classnames
df = pd.read_csv(os.path.join(pathmain, 'know_face_encode.csv'))
encodeListKnow = list(np.array(df))
dfname = pd.read_csv(os.path.join(pathmain, 'know_face_classnames.csv'))
classnames = list(map( lambda x: x[0],  list(np.array(dfname))))
print(classnames)
# read encodeListUnKnow + classnames
dfUnknow = pd.read_csv(os.path.join(pathmain, 'unknow_face_encode.csv'))
encodeListUnknow = list(np.array(dfUnknow))
dfnameCorrect = pd.read_csv(os.path.join(pathmain, 'unknow_face_classnames.csv'))
classnamesCorrect = list(map( lambda x: x[0],  list(np.array(dfnameCorrect))))
print(classnamesCorrect)

def thamdu(name):
    with open(f"/content/drive/MyDrive/Du Lieu PBL/SaveDir/thamdu.csv", "r+") as f:
        myDatalist = f.readlines()
        nameList = []
        timeList = []
        for line in myDatalist:
            entry = line.split(",")
            nameList.append(entry[0])
            timeList.append(entry[1])

        if name not in nameList:
            now = datetime.now()
            dtstring = now.strftime("%H:%M:%S")
            f.writelines(f"\n{name},{dtstring}")
        else:
            # thay doi thoi gian truoc do thanh thoi gian sau
            indexSearch = nameList.index(name)
            timeList[indexSearch] = datetime.now().strftime("%H:%M:%S")
            if indexSearch != len(timeList)-1:
                timeList[indexSearch] += "\n"
            with open(f"/content/drive/MyDrive/Du Lieu PBL/SaveDir/thamdu.csv", "w") as f1:
                for name1, time in zip(nameList, timeList):
                    f1.writelines(f"{name1},{time}")


classnames_guess = []

for encodeFaceNotKnow in encodeListUnknow:
    # matches = face_recognition.compare_faces(encodeListKnow, encodeFaceNotKnow)
    # print(matches)
    faceDis = face_recognition.face_distance(encodeListKnow, encodeFaceNotKnow)
    print(faceDis)
    matchIndex = np.argmin(faceDis)
    if faceDis[matchIndex] < 15:
        print(matchIndex)
        name = classnames[matchIndex].upper()
        thamdu(name)
    else:
        name = "UNKNOW"
    classnames_guess.append(name)

dfname_guess = pd.DataFrame(classnames_guess)

# Save the DataFrame to a CSV file
dfname_guess.to_csv(os.path.join(pathmain, '/unknow_classnames_guess.csv'), index=False)

# calc hiệu suất của thuật toán
exactly = 0
for correct, guess in zip(classnamesCorrect, classnames_guess):
    print(correct, guess)
    # if guess!='UNKNOW':
    #     guess = guess[guess.index('_'):]
    # if correct!= 'UNKNOW':
    #     correct = correct[correct.index('_'):]
    exactly += 1 if guess.upper()==correct.upper() else 0
efficiency = exactly/len(classnamesCorrect) *100
print("efficiency: ", efficiency, "%")

['Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Manh', 'Toan', 'Toan', 'Toan', 'Toan', 'Toan', 'Toan', 'Tri', 'Tri', 'Tri', 'Tri', 'Tri', 'Tri', 'Tri', 'Tri', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Tu', 'Thai', 'Thai', 'Thai', 'Thai', 'Thai', 'Thai', 'Quoc', 'Quoc', 'Quoc', 'Quoc', 'Quoc', 'Quoc', 'Quoc', 'Quoc', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Nghia', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Phuc', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Thuan', 'Tung', 'Tung', 'Tung', 'Tung', 'Tung', 'Tung', 'Tung', 'Can', 'Can', 'Can', 'Can', 'Can', 'Can', 'Can', 'Can', 'Can', 'Can', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Cuong', 'Lich', 'Lich', 'Lich', 'Lich', 'Lich', 'Lich', 'Hung', 'Hung', 'Hung', 'Hung', 'Hung', 'Hung', 'Hung', 'Long', 'Long', 'Long', 'Long', 'Long', 'Long', 'Long', 'Long', 'Huy', 'H